## Notebook for training GPR on DC dataset and generating set of suggestions

**Note: Dataset do not contain any NMC data**

## Batch: 3

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, PandasTools, Fragments, rdMolDescriptors, Descriptors, rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.PandasTools import ChangeMoleculeRendering
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
# Silence non-critical RDKit warnings to minimize unnecessary outputs
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
## import train_test_split from sklearn
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel, Matern, ConstantKernel, DotProduct, PairwiseKernel 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import erf
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%%bash
pwd
ls -ltr

/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/Nat-Comm-R2/active-learning_wo_nmc_data
total 20320


-rw-r--r--@ 1 riteshk  staff  2126450 Apr 17 21:00 active_learning_batch_4.ipynb
-rw-r--r--@ 1 riteshk  staff  1485993 Apr 17 21:00 active_learning_batch_5.ipynb
-rw-r--r--@ 1 riteshk  staff  1474650 Apr 17 21:00 active_learning_batch_6.ipynb
-rw-r--r--@ 1 riteshk  staff  1381340 Apr 17 21:00 active_learning_batch_7.ipynb
-rw-r--r--@ 1 riteshk  staff   964972 Apr 17 21:42 active_learning_batch_1.ipynb
-rw-r--r--@ 1 riteshk  staff   875736 Apr 18 23:45 active_learning_batch_2.ipynb
-rw-r--r--@ 1 riteshk  staff  2078577 Apr 22 08:48 active_learning_batch_3.ipynb


### Reading & standardizing datasets

In [8]:
rem_till_b2 = [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 56, 57, 60, 61, 62, 66, 67, 68, 70, 73, 74, 75, 76, 77, 78, 79, 104, 110, 111, 112, 113, 117, 118, 119]
len(rem_till_b2)

35

In [ ]:
df = pd.read_csv('../../datasets/batch-3/label_data_post_batch2.csv') 
df.drop(index=rem_till_b2, inplace=True)
df.reset_index(inplace=True, drop=True)
df


,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_14,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,0.613072,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,0.301582,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.022233,0.02168,0.022967,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.000573,0.00046,0.000360,0.000407,0.0006,0.000593,0.000447,0.000333,0.0,0.0
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,-0.208395,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)F)(OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.693253,-1.413398,1.016682,-0.620584,0.391537,-0.151953,0.987994,0.129135,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
82,COC(F)(F)C(F)Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.743406,-1.487900,0.912048,-0.821155,0.293529,-0.651772,0.598413,-0.220786,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
83,COC(F)(F)C(F)Cl,[Li+].F[P-](F)(F)(F)(F)F,-0.743406,-1.487900,0.912048,-0.821155,0.293529,-0.651772,0.598413,-0.220786,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
84,COC(F)(F)C(F)Cl,[Li+].O=C1O[B-](F)(F)OC1=O,-0.743406,-1.487900,0.912048,-0.821155,0.293529,-0.651772,0.598413,-0.220786,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0


In [23]:
X = df.iloc[:,2:27] ## PCA-reduced solvent & salt descriptors and other non-molecular features
y = df['norm_capacity_3'] # normalized discharge capacity at 20th cycle (target variable)
std_scale = StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

### Active learning workflow

#### Choose best hyperparameters for each kernel

In [24]:
def negative_log_likelihood_rbf(params):
    noise_level, length_scale, alpha = params
    kernel = RBF(length_scale=length_scale)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rq(params):
    noise_level, length_scale, alpha_k, alpha = params 
    kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha_k)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rbf_expsin(params):
    noise_level, length_scale, periodicity, alpha = params 
    kernel = RBF(length_scale=length_scale) + ExpSineSquared(length_scale=length_scale, periodicity=periodicity)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_matern(params):
    noise_level, length_scale, alpha = params
    kernel = Matern(length_scale=length_scale, nu=1.5)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_pairwise(params):
    noise_level, length_scale, alpha = params
    kernel = PairwiseKernel(metric="polynomial")
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

##### Pairwise kernel

In [25]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_pairwise, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.02000002908341221


##### RationalQuadratic kernel

In [26]:
initial_guess = [0.15, 0.01, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha_k, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha_k, alpha
result = minimize(negative_log_likelihood_rq, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha_k, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha_k:", optimized_alpha_k)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.009999999999935727
Optimized alpha_k: 0.01
Optimized alpha: 0.006291598757812585


##### Matern-3/2 kernel

In [27]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_matern, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.011285249576176298


##### RBF-ExpineSquared kernel

In [28]:
initial_guess = [0.15, 0.01, 1.0, 0.02] # initial guess for noise_level, length_scale, periodicity, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-2, 10.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, periodicity, alpha
result = minimize(negative_log_likelihood_rbf_expsin, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_periodicity, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized periodicity:", optimized_periodicity)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of i

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized periodicity: 1.0
Optimized alpha: 0.019989576882956422


#### Train surrogate models

Note: no need to run again, saved model checkpoints have been provided

In [ ]:
## change all hyperparameters accordingly
optimized_pairwise_kernel = PairwiseKernel(metric="polynomial") + WhiteKernel(noise_level=0.15)
optimized_matern_kernel = Matern(length_scale=0.01, nu=1.5) + WhiteKernel(noise_level=0.15)
optimized_rbfexpsin_kernel = RBF(length_scale=0.01) + ExpSineSquared(length_scale=0.01, periodicity=1.0) + WhiteKernel(noise_level=0.15)
optimized_rq_kernel = RationalQuadratic(length_scale=0.009999999999935727, alpha=0.01) + WhiteKernel(noise_level=0.15)

gpr_models = [GaussianProcessRegressor(kernel=optimized_pairwise_kernel, alpha=0.02000002908341221, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_matern_kernel, alpha=0.011285249576176298, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rq_kernel, alpha=0.006291598757812585, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rbfexpsin_kernel, alpha=0.019989576882956422, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)]

model_names = ['../../models/batch-3/pairwise_batch3_wo_nmc_data.pkl', '../../models/batch-3/matern_batch3_wo_nmc_data.pkl', '../../models/batch-3/rq_batch3_wo_nmc_data.pkl', '../../models/batch-3/rbf-ess_batch3_wo_nmc_data.pkl']
k = 0
for model in gpr_models:
    print("fitting model: ", k)
    model.fit(X_std, y)
    pickle.dump(model, open(model_names[k], 'wb'))
    k += 1

fitting model:  0


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


fitting model:  1
fitting model:  2
fitting model:  3


#### BMA: aggregate predictions

##### Acquisition function (Expected improvement)

In [30]:
## final corrected & verified one to be used
def calc_EI(y_pred, y_pred_un, y_pred_un_uncer, epsilon=0.01):
    y_best = np.max(y_pred)
    EI = []
    explore = []
    exploit = []

    for i in range(len(y_pred_un)):
        if y_pred_un_uncer[i] != 0:
            
            # Calculate the cumulative distribution function (CDF) for the Gaussian distribution
            z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            # z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            cdf_z = 0.5 * (1 + erf(z / np.sqrt(2)))
            pdf_z = np.exp(-0.5 * z**2) / np.sqrt(2 * np.pi)

            # Calculate Expected Improvement
            expected_improvement = y_pred_un_uncer[i] * (z * cdf_z) + y_pred_un_uncer[i] * pdf_z
            exploitation = y_pred_un_uncer[i] * z * cdf_z
            exploration = y_pred_un_uncer[i] * pdf_z
            EI.append(expected_improvement)
            explore.append(exploration)
            exploit.append(exploitation)
        else:
            EI.append(0.0)
    return EI, exploit, explore

In [ ]:
## virtual search space for batch-3 (electrolytes containing solvent combinations tested in batch-2 removed)
path = '/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/data-codes-sharing/datasets'
df_unlabel = pd.read_csv(f'../../datasets/batch-3/virtual_search_space_for_batch3.csv')
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,salt_ecfp_pca_5,salt_ecfp_pca_6,salt_ecfp_pca_7,salt_ecfp_pca_8,salt_ecfp_pca_9,mol_wt_solv,mol_wt_salt,conc_salt_1,theor_capacity,amt_electrolyte
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,73.052764,186.939685,1,150,50
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,114.079313,186.939685,1,150,50
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,116.094963,186.939685,1,150,50
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,70.058995,186.939685,1,150,50
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,72.044939,186.939685,1,150,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999925,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,193.077264,144.001775,1,150,50
999926,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,240.147392,144.001775,1,150,50
999927,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,251.046358,144.001775,1,150,50
999928,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,248.133634,144.001775,1,150,50


In [36]:
X_un = df_unlabel.iloc[:,2:]
X_un_std = std_scale.transform(X_un)
X_un_std = pd.DataFrame(X_un_std, columns=X_un.columns)

##### Calculate model weights & obtained aggregated mean ($\mu^{aggr}$), uncertainty ($\sigma^{aggr}$), & EI ($EI^{aggr}$)

In [ ]:
# Calculate Model Weights using BMA (first order)
model_names = ['../../models/batch-3/pairwise_batch3_wo_nmc_data.pkl', '../../models/batch-3/matern_batch3_wo_nmc_data.pkl', '../../models/batch-3/rq_batch3_wo_nmc_data.pkl', '../../models/batch-3/rbf-ess_batch3_wo_nmc_data.pkl']
model_weights = []
uncertainties = []
predictions = []
y_label_preds = []
for model in model_names:
    gpr = pickle.load(open(model, 'rb'))
    y_un = gpr.predict(X_un_std)
    predictions.append(y_un)
    individual_uncertainties = gpr.predict(X_un_std, return_std=True)[1]
    uncertainties.append(individual_uncertainties)
    likelihoods = norm.pdf(y_un, loc=gpr.predict(X_un_std), scale=individual_uncertainties)
    prior_beliefs = 1.0  # Non-informative prior
    posterior = likelihoods * prior_beliefs
    model_weights.append(posterior / np.sum(posterior))
    y_ = gpr.predict(X_std)
    y_label_preds.append(y_)

In [40]:
df_unlabel['uncertainty_1'] = uncertainties[0]; df_unlabel['uncertainty_2'] = uncertainties[1]; df_unlabel['uncertainty_3'] = uncertainties[2]; df_unlabel['uncertainty_4'] = uncertainties[3]
df_unlabel['prediction_1'] = predictions[0]; df_unlabel['prediction_2'] = predictions[1]; df_unlabel['prediction_3'] = predictions[2]; df_unlabel['prediction_4'] = predictions[3]
df_unlabel['explore_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[2]; df_unlabel['exploit_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[1]
df_unlabel['explore_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[2]; df_unlabel['exploit_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[1]
df_unlabel['explore_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[2]; df_unlabel['exploit_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[1]
df_unlabel['explore_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[2]; df_unlabel['exploit_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[1]
df_unlabel['EI_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[0]; df_unlabel['EI_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[0]; df_unlabel['EI_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[0]; df_unlabel['EI_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[0]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,explore_2,exploit_2,explore_3,exploit_3,explore_4,exploit_4,EI_1,EI_2,EI_3,EI_4
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.014630,-0.012226,0.015212,-0.012715,0.009913,-0.008502,0.001328,0.002404,0.002497,0.001411
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.006470,-0.005649,0.007664,-0.006662,0.002230,-0.002011,0.000564,0.000821,0.001002,0.000220
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.019736,-0.016265,0.012573,-0.010688,0.010396,-0.008950,0.002513,0.003472,0.001884,0.001446
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.007751,-0.006680,0.012092,-0.010206,0.008033,-0.006912,0.000666,0.001071,0.001887,0.001120
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.015317,-0.012782,0.014815,-0.012422,0.022201,-0.018009,0.001380,0.002535,0.002393,0.004192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999925,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.002461,-0.002215,0.004805,-0.004254,0.002250,-0.002030,0.000106,0.000246,0.000551,0.000220
999926,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.017050,-0.014433,0.011397,-0.009796,0.011311,-0.009823,0.000853,0.002617,0.001601,0.001488
999927,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.017037,-0.014391,0.012281,-0.010510,0.012468,-0.010756,0.001431,0.002646,0.001771,0.001712
999928,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.025307,-0.020862,0.012566,-0.010746,0.045042,-0.035335,0.002078,0.004444,0.001820,0.009707


In [41]:
def calc_aggr_uncer(uncer_1, w_1, pred_1, uncer_2, w_2, pred_2, uncer_3, w_3, pred_3, uncer_4, w_4, pred_4):
    uncer = [uncer_1, uncer_2, uncer_3, uncer_4]
    pred = [pred_1, pred_2, pred_3, pred_4]
    weight = [w_1, w_2, w_3, w_4]
    pred_aggr = w_1 * pred_1 + w_2 * pred_2 + w_3 * pred_3 + w_4 * pred_4
    sum = 0
    for i in range(4):
        sum += weight[i] * (uncer[i]**2 + (pred[i] - pred_aggr)**2)
    aggr_uncer = np.sqrt(sum)
    return aggr_uncer

In [42]:
df_unlabel['prediction_aggr'] = df_unlabel['prediction_1'] * model_weights[0] + df_unlabel['prediction_2'] * model_weights[1] + df_unlabel['prediction_3'] * model_weights[2] + df_unlabel['prediction_4'] * model_weights[3]
df_unlabel['uncertainty_aggr'] = calc_aggr_uncer(df_unlabel['uncertainty_1'], model_weights[0], df_unlabel['prediction_1'], df_unlabel['uncertainty_2'], model_weights[1], df_unlabel['prediction_2'], df_unlabel['uncertainty_3'], model_weights[2], df_unlabel['prediction_3'], df_unlabel['uncertainty_4'], model_weights[3], df_unlabel['prediction_4'])
df_unlabel['explore_aggr'] = df_unlabel['explore_1'] * model_weights[0] + df_unlabel['explore_2'] * model_weights[1] + df_unlabel['explore_3'] * model_weights[2] + df_unlabel['explore_4'] * model_weights[3]
df_unlabel['exploit_aggr'] = df_unlabel['exploit_1'] * model_weights[0] + df_unlabel['exploit_2'] * model_weights[1] + df_unlabel['exploit_3'] * model_weights[2] + df_unlabel['exploit_4'] * model_weights[3]
df_unlabel['ratio_aggr'] = df_unlabel['exploit_aggr'] / df_unlabel['explore_aggr']

## 'EI_aggr' is the final rank by which candidate electrolytes are selected for experimental validation
df_unlabel['EI_aggr'] = df_unlabel['EI_1'] * model_weights[0] + df_unlabel['EI_2'] * model_weights[1] + df_unlabel['EI_3'] * model_weights[2] + df_unlabel['EI_4'] * model_weights[3]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.001328,0.002404,0.002497,0.001411,9.873068e-07,0.000690,5.500496e-08,-4.641331e-08,-0.843802,8.591652e-09
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.000564,0.000821,0.001002,0.000220,4.733412e-07,0.000567,2.303636e-08,-2.018946e-08,-0.876417,2.846904e-09
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.002513,0.003472,0.001884,0.001446,6.942832e-07,0.000640,5.865337e-08,-4.920321e-08,-0.838881,9.450155e-09
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.000666,0.001071,0.001887,0.001120,1.172959e-06,0.000717,4.113101e-08,-3.525107e-08,-0.857044,5.879939e-09
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.001380,0.002535,0.002393,0.004192,1.041825e-06,0.000717,6.847203e-08,-5.682399e-08,-0.829886,1.164804e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999925,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.000106,0.000246,0.000551,0.000220,4.254553e-08,0.000529,1.117647e-08,-1.000771e-08,-0.895426,1.168764e-09
999926,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.000853,0.002617,0.001601,0.001488,-3.682009e-08,0.000609,3.849005e-08,-3.308002e-08,-0.859443,5.410028e-09
999927,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.001431,0.002646,0.001771,0.001712,1.173555e-07,0.000597,4.441833e-08,-3.796919e-08,-0.854809,6.449148e-09
999928,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.002078,0.004444,0.001820,0.009707,2.617302e-07,0.000621,7.627498e-08,-6.218716e-08,-0.815302,1.408782e-08


##### Save top 10000 predictions

In [43]:
df_unlabel_ = df_unlabel.copy()
df_unlabel_ = df_unlabel_.sort_values(by='EI_aggr', ascending=False)
df_unlabel_10000 = df_unlabel_.iloc[:10000,:]
df_unlabel_10000

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
149731,COCCOCCOCCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.941841,-0.446580,-0.362501,-0.556506,-0.323321,1.414259,1.014590,0.995675,...,0.041366,0.036027,0.003449,0.138702,1.860884e-06,0.001159,2.940575e-07,-8.785428e-08,-0.298766,2.062032e-07
191184,COCCCN(C)C(=O)CC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.595303,0.426419,0.077841,-0.367198,0.813100,1.088357,1.117794,0.920623,...,0.040167,0.033964,0.003350,0.122820,1.660867e-06,0.001093,2.870429e-07,-1.084073e-07,-0.377669,1.786356e-07
326343,COCCCCCCN(C)C(=O)CN(C)C(=O)CN(C)C(=O)CN(C)C(=O...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.642934,0.583175,-0.193497,0.027452,0.286315,1.477630,0.542843,1.242775,...,0.091143,0.052840,0.004736,0.100400,1.262273e-06,0.001000,3.061273e-07,-1.299172e-07,-0.424389,1.762101e-07
281966,COCCN(C)C(=O)CCCCCCOCCCCCC(=O)N(C)CCC[Si](OC)(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.526972,0.762888,-0.043111,0.091130,-0.036244,1.783612,0.528564,0.863761,...,0.112917,0.060500,0.005424,0.084342,1.141613e-06,0.000972,3.045020e-07,-1.306084e-07,-0.428925,1.738936e-07
295530,COCCN(CCC[Si](OC)(OC)OC)CCN(C)CCC[Si](OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.875932,-0.823209,-0.376278,-0.686554,-0.435530,1.276795,1.090799,0.968467,...,0.062128,0.043523,0.003796,0.095310,1.551820e-06,0.001060,3.012498e-07,-1.291217e-07,-0.428620,1.721281e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640292,COC(=O)CN(CC(=O)N(C)C)C(=O)N(C)CCCN1CCN(C)CC1,[Li+].F[P-](F)(F)(F)(F)F,-0.230539,1.748833,-0.203416,0.678237,0.289340,0.578493,0.577974,1.796076,...,0.026204,0.023597,0.004053,0.035923,4.624710e-07,0.000729,1.871746e-07,-1.324733e-07,-0.707753,5.470131e-08
582807,CN(CCCCCN(CCC(=O)N(C)C)CCC(=O)N(C)C)CCN(CCC(=O...,[Li+].F[P-](F)(F)(F)(F)F,-1.125465,0.271310,-0.720026,0.438961,1.278918,0.658953,-0.481483,0.653467,...,0.027399,0.022785,0.004084,0.033133,5.011543e-07,0.000731,1.872501e-07,-1.325547e-07,-0.707902,5.469541e-08
183882,CN(C)C(=O)N(C)CCN(C)C(=O)CCCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.144357,0.460640,-0.671151,0.247343,1.469207,0.628446,-0.594683,0.663495,...,0.021352,0.017765,0.003296,0.026481,8.970500e-07,0.000782,1.860435e-07,-1.313485e-07,-0.706010,5.469491e-08
56903,CN(C)CCOCCOC=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.811062,-0.712121,-0.465502,-0.581470,0.542709,0.527661,0.291054,0.871578,...,0.011825,0.013868,0.002460,0.026555,1.399802e-06,0.000893,1.826545e-07,-1.279601e-07,-0.700558,5.469437e-08


In [45]:
df_unlabel_uniq = df_unlabel_10000.drop_duplicates(subset=['solv_comb_sm'], keep='first') ## only keeping unique solvent combinations for selection purposes; these compounds were manually searched in emolecules to finf purchasable compounds
df_unlabel_uniq['solv_comb_sm'] = df_unlabel_uniq['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_unlabel_uniq[['solv_comb_sm', 'salt_comb_sm', 'prediction_aggr', 'uncertainty_aggr', 'explore_aggr', 'exploit_aggr', 'ratio_aggr', 'EI_aggr']].to_csv('../datasets/batch-3/top_10000_suggestions_batch3_uniq_solvents_wo_nmc_data.csv', index=False)
df_unlabel_uniq

/var/folders/s5/lmr5dh0n4bsgnpxrzwlts5lw0000gn/T/ipykernel_1489/1025700394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabel_uniq['solv_comb_sm'] = df_unlabel_uniq['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))


,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
149731,COCCOCCOCCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.941841,-0.446580,-0.362501,-0.556506,-0.323321,1.414259,1.014590,0.995675,...,0.041366,0.036027,0.003449,0.138702,1.860884e-06,0.001159,2.940575e-07,-8.785428e-08,-0.298766,2.062032e-07
191184,COCCCN(C)C(=O)CC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.595303,0.426419,0.077841,-0.367198,0.813100,1.088357,1.117794,0.920623,...,0.040167,0.033964,0.003350,0.122820,1.660867e-06,0.001093,2.870429e-07,-1.084073e-07,-0.377669,1.786356e-07
326343,COCCCCCCN(C)C(=O)CN(C)C(=O)CN(C)C(=O)CN(C)C(=O...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.642934,0.583175,-0.193497,0.027452,0.286315,1.477630,0.542843,1.242775,...,0.091143,0.052840,0.004736,0.100400,1.262273e-06,0.001000,3.061273e-07,-1.299172e-07,-0.424389,1.762101e-07
281966,COCCN(C)C(=O)CCCCCCOCCCCCC(=O)N(C)CCC[Si](OC)(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.526972,0.762888,-0.043111,0.091130,-0.036244,1.783612,0.528564,0.863761,...,0.112917,0.060500,0.005424,0.084342,1.141613e-06,0.000972,3.045020e-07,-1.306084e-07,-0.428925,1.738936e-07
295530,COCCN(CCC[Si](OC)(OC)OC)CCN(C)CCC[Si](OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.875932,-0.823209,-0.376278,-0.686554,-0.435530,1.276795,1.090799,0.968467,...,0.062128,0.043523,0.003796,0.095310,1.551820e-06,0.001060,3.012498e-07,-1.291217e-07,-0.428620,1.721281e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640292,COC(=O)CN(CC(=O)N(C)C)C(=O)N(C)CCCN1CCN(C)CC1,[Li+].F[P-](F)(F)(F)(F)F,-0.230539,1.748833,-0.203416,0.678237,0.289340,0.578493,0.577974,1.796076,...,0.026204,0.023597,0.004053,0.035923,4.624710e-07,0.000729,1.871746e-07,-1.324733e-07,-0.707753,5.470131e-08
582807,CN(CCCCCN(CCC(=O)N(C)C)CCC(=O)N(C)C)CCN(CCC(=O...,[Li+].F[P-](F)(F)(F)(F)F,-1.125465,0.271310,-0.720026,0.438961,1.278918,0.658953,-0.481483,0.653467,...,0.027399,0.022785,0.004084,0.033133,5.011543e-07,0.000731,1.872501e-07,-1.325547e-07,-0.707902,5.469541e-08
183882,CN(C)C(=O)N(C)CCN(C)C(=O)CCCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.144357,0.460640,-0.671151,0.247343,1.469207,0.628446,-0.594683,0.663495,...,0.021352,0.017765,0.003296,0.026481,8.970500e-07,0.000782,1.860435e-07,-1.313485e-07,-0.706010,5.469491e-08
56903,CN(C)CCOCCOC=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.811062,-0.712121,-0.465502,-0.581470,0.542709,0.527661,0.291054,0.871578,...,0.011825,0.013868,0.002460,0.026555,1.399802e-06,0.000893,1.826545e-07,-1.279601e-07,-0.700558,5.469437e-08


In [49]:
df_unlabel_uniq_ = pd.read_csv(f'{path}/batch-3/top_10000_suggestions_batch3_uniq_solvents.csv')
df_unlabel_uniq_

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,conc_salt_2,theor_capacity,amt_electrolyte,pressure_type,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.371209,0.423985,0.087168,-0.390568,1.106635,0.478028,0.524065,0.414960,...,0.0,150,50,2,9.818468e-07,0.000796,1.144290e-07,-8.759247e-08,-3.263929,2.683651e-08
1,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.431278,0.458372,0.033057,-0.526323,1.347255,0.546517,0.808674,0.145876,...,0.0,150,50,2,9.729763e-07,0.000797,1.184382e-07,-9.013896e-08,-3.185212,2.829920e-08
2,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.369815,0.425456,0.086003,-0.388726,1.105710,0.477714,0.523496,0.416414,...,0.0,150,50,2,1.031936e-06,0.000794,9.934253e-08,-7.795002e-08,-3.643800,2.139251e-08
3,COC(=O)C(F)(F)C(F)(F)C(F)(F)C(=O)N(C)CCC[Si](O...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.538909,0.674474,0.272739,-0.443989,1.058057,0.533427,0.546245,0.314937,...,0.0,150,50,2,1.027699e-06,0.000792,9.836540e-08,-7.733640e-08,-3.677607,2.102900e-08
4,CO[Si](CCCCN(C)C(=O)N(C(=O)N(C)C)C(=O)N(C)CCCC...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.370517,0.607333,-0.126657,-0.230992,0.625624,0.741056,-0.063577,0.746594,...,0.0,150,50,2,8.526140e-07,0.000795,1.457390e-07,-1.048731e-07,-2.566271,4.086594e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8949,CCCN(C)CCCCOCCOC,[Li+].O=C1O[B-](F)(F)OC1=O,-1.200232,-0.716220,-0.632898,0.237957,-0.464196,1.822570,0.894457,0.749287,...,0.0,150,50,2,9.301365e-07,0.000749,7.381771e-08,-5.988471e-08,-4.298050,1.393300e-08
8950,COCCN(C)CCC[Si](COC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-1.029200,-0.997617,-0.280489,-0.503384,-0.357378,1.088322,1.093044,1.384092,...,0.0,150,50,2,9.066336e-07,0.000738,6.721053e-08,-5.494478e-08,-4.479527,1.226576e-08
8951,COCCOCCCN(C)CCCCl,[Li+].O=C1O[B-](F)(F)OC1=O,-0.951036,-0.576891,-0.254395,-0.465784,-0.325791,1.397665,1.032146,0.948021,...,0.0,150,50,2,9.882002e-07,0.000762,6.936447e-08,-5.653333e-08,-4.405945,1.283115e-08
8952,CCCCOCCN(CCC[Si](OC)(OC)OC)CCOC,[Li+].O=C1O[B-](F)(F)OC1=O,-1.164254,-1.313408,-0.158492,0.408141,-0.805483,1.630553,0.830279,0.463970,...,0.0,150,50,2,9.512937e-07,0.000759,7.802918e-08,-6.290676e-08,-4.159836,1.512242e-08


In [58]:
df_unlabel_uniq_['solv_comb_sm'] = df_unlabel_uniq_['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm', 'salt_comb_sm'], how='inner') ## unique electrolytes
# df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm'], how='inner') ## unique solvents
df_sugg_comm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0_x,solv_ecfp_pca_1_x,solv_ecfp_pca_2_x,solv_ecfp_pca_3_x,solv_ecfp_pca_4_x,solv_ecfp_pca_5_x,solv_ecfp_pca_6_x,solv_ecfp_pca_7_x,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr_y,uncertainty_aggr_y,explore_aggr_y,exploit_aggr_y,ratio_aggr_y,EI_aggr_y
0,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.371209,0.423985,0.087168,-0.390568,1.106635,0.478028,0.524065,0.414960,...,0.036122,0.026376,0.003677,0.040357,9.877187e-07,0.000838,2.273323e-07,-1.463693e-07,-0.643856,8.096296e-08
1,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.371209,0.423985,0.087168,-0.390568,1.106635,0.478028,0.524065,0.414960,...,0.036122,0.026376,0.003677,0.040357,9.877187e-07,0.000838,2.273323e-07,-1.463693e-07,-0.643856,8.096296e-08
2,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.431278,0.458372,0.033057,-0.526323,1.347255,0.546517,0.808674,0.145876,...,0.036874,0.027642,0.003796,0.018827,8.584368e-07,0.000794,2.022637e-07,-1.361945e-07,-0.673351,6.606923e-08
3,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.431278,0.458372,0.033057,-0.526323,1.347255,0.546517,0.808674,0.145876,...,0.036874,0.027642,0.003796,0.018827,8.584368e-07,0.000794,2.022637e-07,-1.361945e-07,-0.673351,6.606923e-08
4,CO[Si](CCCN(C)C(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.369815,0.425456,0.086003,-0.388726,1.105710,0.477714,0.523496,0.416414,...,0.029829,0.022961,0.003297,0.031689,1.046205e-06,0.000838,2.133572e-07,-1.419803e-07,-0.665458,7.137696e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,COC(=O)CCOCCN(C)C(=O)N(C)CC(=O)N(C)C,[Li+].F[P-](F)(F)(F)(F)F,-1.699380,1.207882,0.255854,-0.379775,0.372623,0.945207,-0.023897,1.010628,...,0.021614,0.019496,0.003281,0.032465,7.249466e-07,0.000754,1.868696e-07,-1.314093e-07,-0.703214,5.546028e-08
7388,COC(=O)CN(CC(=O)N(C)C)C(=O)N(C)CCCN1CCN(C)CC1,[Li+].F[P-](F)(F)(F)(F)F,-0.230539,1.748833,-0.203416,0.678237,0.289340,0.578493,0.577974,1.796076,...,0.026204,0.023597,0.004053,0.035923,4.624710e-07,0.000729,1.871746e-07,-1.324733e-07,-0.707753,5.470131e-08
7389,CN(CCOCCOCCCC(=O)N(C)N(C)C)C(=O)COC=O,[Li+].F[P-](F)(F)(F)(F)F,-1.110739,0.553541,-0.396562,0.087599,0.746942,1.198019,-0.023412,0.467306,...,0.018725,0.017203,0.003046,0.038618,7.848448e-07,0.000768,1.878933e-07,-1.302467e-07,-0.693195,5.764663e-08
7390,CCOC(=O)N(C)CCN(C)C(=O)N(C)COCCOC,[Li+].F[P-](F)(F)(F)(F)F,-1.714921,0.617833,-0.042087,-0.150252,0.191541,1.096016,0.385625,0.895118,...,0.019039,0.017308,0.002761,0.031831,8.665336e-07,0.000779,1.856103e-07,-1.298483e-07,-0.699575,5.576201e-08


### Check how many common solvents in third batch of labeled dataset

In [ ]:
df_label_all = pd.read_csv('../../datasets/label_all_ecfp_pca_add_feat_incl_b7_090824.csv')
df_label_all

,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.02168,0.022967,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.00046,0.000360,0.000407,0.0006,0.000593,0.000447,0.000333,0.0,0.0,0.0
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,CCOCCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
204,COCCCCOCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
205,COCCCOCC(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
206,COCCCOCC(F)(F)C(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0


In [51]:
df_b3 = df_label_all.loc[df_label_all['expt_test'] == 3]
df_b3['solv_comb_sm'] = df_b3['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_b3

/var/folders/s5/lmr5dh0n4bsgnpxrzwlts5lw0000gn/T/ipykernel_1489/1564948772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b3['solv_comb_sm'] = df_b3['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))


,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
121,COCCOCCOCC(F)(F)C(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-1.190545,-1.129056,1.030187,-0.668465,-0.151808,-0.097468,1.281233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
122,CCCCOCCOCCOCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
123,CCCCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-1.308256,-1.863069,0.558941,1.032228,-0.733941,1.279644,0.535023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
124,CCOC(=O)CN(C)CCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-1.618370,0.619338,0.080507,0.000244,0.093421,0.996731,0.388905,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
125,COCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
126,COCCN(C)C,[Li+].O=C1O[B-](F)(F)OC1=O,3.0,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
127,CCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-0.761120,-1.609598,0.396423,0.137856,0.181540,0.205658,0.102161,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
128,CCOCCCl,[Li+].F[P-](F)(F)(F)(F)F,3.0,-0.761120,-1.609598,0.396423,0.137856,0.181540,0.205658,0.102161,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
129,CCOCCCl,[Li+].O=C1O[B-](F)(F)OC1=O,3.0,-0.761120,-1.609598,0.396423,0.137856,0.181540,0.205658,0.102161,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
130,COCCN(C)S(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,3.0,-0.997594,-0.587529,-0.084673,-0.766005,-0.003629,0.705357,0.825030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [52]:
uniq_sm_b3 = df_b3['solv_comb_sm'].unique()
uniq_sm_b3 = pd.DataFrame(uniq_sm_b3, columns=['solv_comb_sm'])
uniq_sm_b3

,solv_comb_sm
0,COCCOCCOCC(F)(F)C(F)F
1,CCCCOCCOCCOCCOC
2,CCCCCCOC
3,CCOC(=O)CN(C)CCOC
4,COCCN(C)C
5,CCOCCCl
6,COCCN(C)S(C)(=O)=O


In [53]:
df_comm = df_unlabel_uniq.merge(uniq_sm_b3, on='solv_comb_sm', how='right')
df_comm.dropna(inplace=True)
df_comm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
1,CCCCOCCOCCOCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,0.057886,...,0.014707,0.013376,0.001820,0.025234,0.000002,0.001033,1.992596e-07,-1.340170e-07,-0.672575,6.524265e-08
3,CCOC(=O)CN(C)CCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.618370,0.619338,0.080507,0.000244,0.093421,0.996731,0.388905,0.901693,...,0.026976,0.021815,0.003322,0.039199,0.000001,0.000930,2.301984e-07,-1.462529e-07,-0.635334,8.394553e-08
4,COCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,1.288009,...,0.023248,0.024307,0.003187,0.053633,0.000002,0.000999,2.464421e-07,-1.461070e-07,-0.592865,1.003351e-07
6,COCCN(C)S(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.997594,-0.587529,-0.084673,-0.766005,-0.003629,0.705357,0.825030,1.169960,...,0.024955,0.025030,0.002638,0.039113,0.000002,0.001006,2.405141e-07,-1.477311e-07,-0.614230,9.278301e-08


In [ ]:
df_comm.to_csv('../../datasets/batch-3/labeled_batch3_uniq_solvents_wo_nmc_data.csv', index=False)

In [64]:
df_comm_label = df_unlabel_uniq.merge(df_b3, on='solv_comb_sm', how='left', suffixes=('', '_drop'))
df_comm_label.dropna(inplace=True)
df_comm_label = df_comm_label.loc[:, ~df_comm_label.columns.str.endswith('_drop')]
df_comm_label

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
1060,COCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,1.288009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1061,COCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,1.288009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1547,COCCN(C)S(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.997594,-0.587529,-0.084673,-0.766005,-0.003629,0.705357,0.825030,1.169960,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2301,CCOC(=O)CN(C)CCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.618370,0.619338,0.080507,0.000244,0.093421,0.996731,0.388905,0.901693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5579,CCCCOCCOCCOCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,0.057886,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [ ]:
df_comm_label.to_csv('../../datasets/batch-3/labeled_batch3_all_wo_nmc_data.csv', index=False)

In [55]:
df_uncomm = df_unlabel_uniq.merge(uniq_sm_b3, on='solv_comb_sm', how='left')
df_uncomm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,COCCOCCOCCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.941841,-0.446580,-0.362501,-0.556506,-0.323321,1.414259,1.014590,0.995675,...,0.041366,0.036027,0.003449,0.138702,1.860884e-06,0.001159,2.940575e-07,-8.785428e-08,-0.298766,2.062032e-07
1,COCCCN(C)C(=O)CC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.595303,0.426419,0.077841,-0.367198,0.813100,1.088357,1.117794,0.920623,...,0.040167,0.033964,0.003350,0.122820,1.660867e-06,0.001093,2.870429e-07,-1.084073e-07,-0.377669,1.786356e-07
2,COCCCCCCN(C)C(=O)CN(C)C(=O)CN(C)C(=O)CN(C)C(=O...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.642934,0.583175,-0.193497,0.027452,0.286315,1.477630,0.542843,1.242775,...,0.091143,0.052840,0.004736,0.100400,1.262273e-06,0.001000,3.061273e-07,-1.299172e-07,-0.424389,1.762101e-07
3,COCCN(C)C(=O)CCCCCCOCCCCCC(=O)N(C)CCC[Si](OC)(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.526972,0.762888,-0.043111,0.091130,-0.036244,1.783612,0.528564,0.863761,...,0.112917,0.060500,0.005424,0.084342,1.141613e-06,0.000972,3.045020e-07,-1.306084e-07,-0.428925,1.738936e-07
4,COCCN(CCC[Si](OC)(OC)OC)CCN(C)CCC[Si](OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.875932,-0.823209,-0.376278,-0.686554,-0.435530,1.276795,1.090799,0.968467,...,0.062128,0.043523,0.003796,0.095310,1.551820e-06,0.001060,3.012498e-07,-1.291217e-07,-0.428620,1.721281e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9469,COC(=O)CN(CC(=O)N(C)C)C(=O)N(C)CCCN1CCN(C)CC1,[Li+].F[P-](F)(F)(F)(F)F,-0.230539,1.748833,-0.203416,0.678237,0.289340,0.578493,0.577974,1.796076,...,0.026204,0.023597,0.004053,0.035923,4.624710e-07,0.000729,1.871746e-07,-1.324733e-07,-0.707753,5.470131e-08
9470,CN(CCCCCN(CCC(=O)N(C)C)CCC(=O)N(C)C)CCN(CCC(=O...,[Li+].F[P-](F)(F)(F)(F)F,-1.125465,0.271310,-0.720026,0.438961,1.278918,0.658953,-0.481483,0.653467,...,0.027399,0.022785,0.004084,0.033133,5.011543e-07,0.000731,1.872501e-07,-1.325547e-07,-0.707902,5.469541e-08
9471,CN(C)C(=O)N(C)CCN(C)C(=O)CCCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.144357,0.460640,-0.671151,0.247343,1.469207,0.628446,-0.594683,0.663495,...,0.021352,0.017765,0.003296,0.026481,8.970500e-07,0.000782,1.860435e-07,-1.313485e-07,-0.706010,5.469491e-08
9472,CN(C)CCOCCOC=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.811062,-0.712121,-0.465502,-0.581470,0.542709,0.527661,0.291054,0.871578,...,0.011825,0.013868,0.002460,0.026555,1.399802e-06,0.000893,1.826545e-07,-1.279601e-07,-0.700558,5.469437e-08


In [56]:
uniq_sm_comm = df_comm['solv_comb_sm'].unique()
for i in range(len(uniq_sm_b3)):
    if uniq_sm_b3['solv_comb_sm'][i] not in uniq_sm_comm:
        print(uniq_sm_b3['solv_comb_sm'][i])

COCCOCCOCC(F)(F)C(F)F
CCCCCCOC
CCOCCCl
